In [1]:
import numpy as np
import time
import csv
import gillespy2
from gillespy2 import Model, Species, Parameter, Reaction, Event, \
                      EventTrigger, EventAssignment, RateRule, \
                      AssignmentRule, FunctionDefinition, TimeSpan
# from gillespy2 import ODECSolver
from gillespy2 import SSACSolver
# from gillespy2 import CLESolver
# from gillespy2 import TauLeapingCSolver
# from gillespy2 import TauHybridCSolver

In [4]:
class PredatorPrey1(Model):
    def __init__(self, tspan):
        Model.__init__(self, name="Predator-Prey(1)")
        self.volume = 1

        # Parameters
        r_birth = Parameter(name="r_birth", expression="10")
        f_birth = Parameter(name="f_birth", expression="0.01")
        f_death = Parameter(name="f_death", expression="10")
        self.add_parameter([r_birth, f_birth, f_death])

        # Variables
        rabbit = Species(name="rabbit", initial_value=1000, mode="discrete")
        fox = Species(name="fox", initial_value=1000, mode="discrete")
        self.add_species([rabbit, fox])

        # Reactions
        rabbit_birth = Reaction(name="rabbit_birth", reactants={'rabbit': 1}, products={'rabbit': 2}, rate="r_birth")
        fox_birth = Reaction(name="fox_birth", reactants={'rabbit': 1, 'fox': 1}, products={'fox': 2}, rate="f_birth")
        fox_death = Reaction(name="fox_death", reactants={'fox': 1}, products={}, rate="f_death")
        self.add_reaction([rabbit_birth, fox_birth, fox_death])

        # Timespan
        self.timespan(tspan)

In [5]:
def configure_simulation(n, tspan_n):
    tspan = TimeSpan(np.arange(0, tspan_n, 0.01))
    model = PredatorPrey1(tspan)
    solver = SSACSolver(model=model)
    kwargs = {
        "solver":solver,
        "number_of_trajectories":n,
        "timeout":20,
        # "seed":None,
        # "tau_tol":0.03,
        # "integrator_options":{'rtol': 0.001, 'atol': 1e-06},
    }
    return kwargs

In [6]:
# What value(s) do you want to extract from the simulation trajectory
# i = 0: fox; i = 1: rabbit
def population_at_last_timepoint(c, res, i):
    if c.verbose:
        print(f'population_at_last_timepoint {c.variable_of_interest[i]}={res[c.variable_of_interest[i]][-1]}')
        #print(res)
        #print(res[c.variable_of_interest])
    return res[c.variable_of_interest[i]][-1]

In [7]:
# How do we combine the values from multiple trajectores
def average_of_ensemble(c, data):
    a = np.average(data)
    if c.verbose:
        print(f'average_of_ensemble = {a}')
    return a

In [8]:
class ParameterSweep3D():
    
    def write_to_file(c, filename):
        with open(filename, 'a', encoding='UTF8', newline = "") as f:
            writer = csv.writer(f)
            #row = ["runtime", "The number of rabbits", c.p1, c.p2, c.p3, "number_of_trajectories"]
            #writer.writerow(row)
            for i in range(len(c.para)):
                row = []
                row.append(c.runtime[i])
                row.append(c.data_rabbit[i])
                row.append(c.data_fox[i])
                for j in range(len(c.para[0])):
                    row.append(c.para[i][j])
                row.append(c.number_of_trajectories)
                row.append(c.tspan_n)
                writer.writerow(row)
                
    def plot(c):
        from matplotlib import pyplot as plt
        from mpl_toolkits.axes_grid1 import make_axes_locatable
        fig, ax = plt.subplots(figsize=(8, 8))
        plt.imshow(c.data)
        ax.set_xticks(np.arange(c.data.shape[1]) + 0.5, minor=False)
        ax.set_yticks(np.arange(c.data.shape[0]) + 0.5, minor=False)
        plt.title(f'Parameter Sweep - Variable: {c.variable_of_interest}')
        ax.set_xticklabels(c.p1_range, minor=False, rotation=90)
        ax.set_yticklabels(c.p2_range, minor=False)
        ax.set_xlabel(c.p1, fontsize=16, fontweight='bold')
        ax.set_ylabel(c.p2, fontsize=16, fontweight='bold')
        divider = make_axes_locatable(ax)
        cax = divider.append_axes('right', size='5%', pad=0.2)
        _ = plt.colorbar(ax=ax, cax=cax)


    def plotplotly(c, return_plotly_figure=False):
        from plotly.offline import init_notebook_mode, iplot
        import plotly.graph_objs as go

        xaxis_ticks = c.p1_range
        yaxis_ticks = c.p2_range

        trace_list = [go.Heatmap(z=c.data, x=xaxis_ticks, y=yaxis_ticks)]
        title = dict(text=f'<b>Parameter Sweep - Variable: {c.variable_of_interest}</b>', x=0.5)
        xaxis_label = dict(title=f'<b>{c.p1}</b>')
        yaxis_label = dict(title=f'<b>{c.p2}</b>')

        layout = go.Layout(title=title, xaxis=xaxis_label, yaxis=yaxis_label)

        fig = dict(data=trace_list, layout=layout)

        if return_plotly_figure:
            return fig
        iplot(fig)

In [9]:
def run(c, verbose=False):
    c.verbose = verbose
    fn = c.feature_extraction
    ag = c.ensemble_aggragator
    data_rabbit = np.zeros((len(c.p1_range), len(c.p2_range), len(c.p3_range)))
    data_fox = np.zeros((len(c.p1_range), len(c.p2_range), len(c.p3_range)))
    runtime = np.zeros((len(c.p1_range), len(c.p2_range), len(c.p3_range)))
    para = []
    c.number_of_trajectories = c.kwargs["number_of_trajectories"]
    c.timeout = c.kwargs["timeout"]
    for i, v1 in enumerate(c.p1_range):
        for j, v2 in enumerate(c.p2_range):
            for u, v3 in enumerate(c.p3_range):
                if c.verbose:
                    print(f'running {c.p1}={v1}, {c.p2}={v2}, {c.p3}={v3}')
                        #para = np.append(para, [[v1, v2]], axis = 0)
                para.append([v1, v2, v3])
                start = time.time()
                if(c.number_of_trajectories > 1):
                    tmp_results = c.model.run(**c.kwargs, variables={c.p1:v1, c.p2:v2, c.p3:v3})
                    data_fox[i, j, u] = ag(c,[fn(c,x,0) for x in tmp_results])
                    data_rabbit[i, j, u] = ag(c,[fn(c,x,1) for x in tmp_results])
                        
                else:
                    tmp_results = c.model.run(**c.kwargs, variables={c.p1:v1, c.p2:v2, c.p3:v3})
                    data_fox[i, j, u] = c.feature_extraction(c,tmp_results,0)
                    data_rabbit[i, j, u] = c.feature_extraction(c,tmp_results,1)
                end = time.time()
                if c.verbose:
                    print(f"running time: {end-start}")
                runtime[i, j, u] = end - start
                        
    c.data_fox = data_fox.flatten()
    c.data_rabbit = data_rabbit.flatten()
    c.runtime = runtime.flatten()
    c.para = np.array(para)
    return c

In [10]:
class ParameterSweepConfig3D(ParameterSweep3D):
    def __init__(self, min_num_1, max_num_1, min_num_2, max_num_2, min_num_3, max_num_3, p1_steps, p2_steps, p3_steps, tspan_n, kwargs):
        self.min_num_1 = min_num_1
        self.max_num_1 = max_num_1
        self.min_num_2 = min_num_2
        self.max_num_2 = max_num_2
        self.min_num_3 = min_num_3
        self.max_num_3 = max_num_3
        self.kwargs = kwargs
    
        # What class defines the GillesPy2 model
        tspan = TimeSpan(np.arange(0, tspan_n, 0.01))
        self.tspan_n = tspan_n
        self.model = PredatorPrey1(tspan)
        #self.model = PredatorPrey1()
        # ENTER PARAMETER 1 HERE
        self.p1 = 'r_birth'
        # ENTER PARAMETER 2 HERE
        self.p2 = 'f_birth'
        # ENTER PARAMETER 3 HERE
        self.p3 = 'f_death'
        # ENTER START VALUE FOR P1 RANGE HERE
        self.p1_min = self.min_num_1 * float(eval(self.model.get_parameter(self.p1).expression))
        # ENTER END VALUE FOR P1 RANGE HERE
        self.p1_max = self.max_num_1 * float(eval(self.model.get_parameter(self.p1).expression))
        # ENTER THE NUMBER OF STEPS FOR P1 HERE
        self.p1_steps = p1_steps
        self.p1_range = np.linspace(self.p1_min, self.p1_max, self.p1_steps)
        # ENTER START VALUE FOR P2 RANGE HERE
        self.p2_min = self.min_num_2 * float(eval(self.model.get_parameter(self.p2).expression))
        # ENTER END VALUE FOR P2 RANGE HERE
        self.p2_max = self.max_num_2 * float(eval(self.model.get_parameter(self.p2).expression))
        # ENTER THE NUMBER OF STEPS FOR P2 HERE
        self.p2_steps = p2_steps
        self.p2_range = np.linspace(self.p2_min, self.p2_max, self.p2_steps)

        # ENTER START VALUE FOR P3 RANGE HERE
        self.p3_min = self.min_num_3 * float(eval(self.model.get_parameter(self.p3).expression))
        # ENTER END VALUE FOR P3 RANGE HERE
        self.p3_max = self.max_num_3 * float(eval(self.model.get_parameter(self.p3).expression))
        # ENTER THE NUMBER OF STEPS FOR P3 HERE
        self.p3_steps = p3_steps
        self.p3_range = np.linspace(self.p3_min, self.p3_max, self.p3_steps)

        # ENTER VARIABLE OF INTEREST HERE
        self.variable_of_interest = ['fox','rabbit']
        # What feature of the simulation are we examining
        self.feature_extraction = population_at_last_timepoint
        # for ensemble resutls: how do we aggreggate the values
        self.ensemble_aggragator = average_of_ensemble

In [12]:
with open("fox_rabbit_test.csv", 'w', encoding='UTF8', newline = "") as f:
        writer = csv.writer(f)
        row = ["runtime", "The number of rabbits", "The number of foxes", "r_birth", "f_birth", "f_death", "number_of_trajectories", "Timespan"]
        writer.writerow(row)

In [11]:
kwargs3D = configure_simulation(3, 20)
psnew = ParameterSweepConfig3D(1, 1.5, 1, 1.5, 1, 1.5, 5, 5, 5, 20, kwargs3D)
result = run(psnew)

In [17]:
result.write_to_file("fox_rabbit_test.csv")